#Load Dataset

In [6]:
!wget -O LLM-ReDial-2024.zip "https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1"
!unzip LLM-ReDial-2024.zip
!unzip LLM_Redial.zip
!rm LLM_Redial.zip
!rm LLM-ReDial-2024.zip
!rm -rf __MACOSX

--2024-10-14 21:33:15--  https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc194e716692cd4efde5b4d03b15.dl.dropboxusercontent.com/cd/0/inline/CcfldplB_7B6gZSayU2_avTFMWwzKFMxjCdxfr-gl1SSnemWoaqXpHLw0qgjA2f9vQHxxV66vN19l42_vdWgLsnrjYg-sP_WtXpd4zjX1Mgwie-tOnuUZ2vEZTwuBeJO8_c/file?dl=1# [following]
--2024-10-14 21:33:16--  https://uc194e716692cd4efde5b4d03b15.dl.dropboxusercontent.com/cd/0/inline/CcfldplB_7B6gZSayU2_avTFMWwzKFMxjCdxfr-gl1SSnemWoaqXpHLw0qgjA2f9vQHxxV66vN19l42_vdWgLsnrjYg-sP_WtXpd4zjX1Mgwie-tOnuUZ2vEZTwuBeJO8_c/file?dl=1
Resolving uc194e716692cd4efde5b4d03b15.dl.dropboxusercontent.com (uc194e716692cd4efde5b4d03b15.dl.dropboxusercontent.com)... 162.125.81.15, 2620:100:6031:15::

#Load HF model

In [3]:
from huggingface_hub import notebook_login

notebook_login()



In [1]:
!pip install transformers[torch]
!pip install bitsandbytes
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 14.1 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [4]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [46]:
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
def generate_prompt(data):
  question = data['question']
  ref = data['references']

  chat = [
      {"role": "user", "content": f"Answer the Question: {question} Considering this information: {ref}"},
    ]

  data['prompt'] = tokenizer.apply_chat_template(chat, tokenize=False)
  return data

#dataset['test'] = dataset['test'].map(generate_prompt)

In [43]:
def generate_zeroshot_prompt(likes, dislikes):
  likes_str = "\n".join(likes)
  dislikes_str = "\n".join(dislikes)

  chat = [
    {"role": "user", "content": f"I've liked these movies: {likes_str}. I've disliked these movies: {dislikes_str}. Recommend a new movie for me to watch."}
  ]
  return tokenizer.apply_chat_template(chat, tokenize=False)

In [47]:
from Tools import read_user_data
path_movie_data = "/content/Movie/final_data.jsonl"

In [34]:
import json
item_map_path = "/content/Movie/item_map.json"
item_map = json.load(open(item_map_path,'r'))
user_map_path = "/content/Movie/user_ids.json"
user_map = json.load(open(user_map_path,'r'))

In [36]:
import random
user_ids = list(user_map.keys())
sample_percent = 0.1 # 10%

sample_size = int(sample_percent * len(user_ids))
sampled_users = random.sample(user_ids,sample_size)

In [48]:
prompts = []
ground_truth = []
for user_id in sampled_users:
  user_data = read_user_data(path_movie_data, user_id)
  likes = user_data['Conversation'][0]['conversation_1']['user_likes']
  likes_decode = []
  for movie in likes:
    likes_decode.append(item_map[movie])
  dislikes = user_data['Conversation'][0]['conversation_1']['user_dislikes']
  dislikes_decode = []
  for movie in dislikes:
    dislikes_decode.append(item_map[movie])
  rec_item = user_data['Conversation'][0]['conversation_1']['rec_item']
  rec_item_decode = item_map[rec_item[0]]
  prompt = generate_zeroshot_prompt(likes_decode,dislikes_decode)
  prompts.append(prompt)
  ground_truth.append(rec_item_decode)



In [53]:
answers = []
for prompt in prompts:
  ans = text_generator(prompt, max_new_tokens=30)
  answers.append(ans[0]['generated_text'])


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [54]:
for answer in answers:
  print(answer)


<s>[INST] I've liked these movies: I Can Do Bad All By Myself
The Hunger Games (2 Disc) [Region 2] [UK Import]
Arena. I've disliked these movies: . Recommend a new movie for me to watch.[/INST] Based on your preference for action, drama, and a bit of suspense, I would recommend "Mad Max: Fury Road". This movie is
<s>[INST] I've liked these movies: . I've disliked these movies: The Killer Inside Me [Blu-ray]
Apollo 18. Recommend a new movie for me to watch.[/INST] Based on your preferences, I would recommend "No Country for Old Men" directed by the Coen Brothers. It is a suspenseful and crit
<s>[INST] I've liked these movies: Boy Wonder. I've disliked these movies: . Recommend a new movie for me to watch.[/INST] Based on your preference for "Boy Wonder" and disliking unspecified movies, I would recommend "The Grand Budapest Hotel" directed by
<s>[INST] I've liked these movies: . I've disliked these movies: Gods and Generals VHS
The Matrix Reloaded
Lost Highway (Widescreen Edition). Reco